# Оценка линейного искажающего оператора в задаче восстановления изображений
Время работы 6:22

In [ ]:
import numpy as np
# import matplotlib.pyplot as plt

from scipy.signal import convolve2d
from skimage import color, data, restoration, img_as_float, io
from skimage.measure import compare_psnr
from functools import partial
from scipy.optimize import minimize

from util import *
from richardson_lucy import *

In [ ]:
prefix = "curved-tnc"

In [ ]:
astro = color.rgb2gray(data.astronaut())
liftingbody = img_as_float(io.imread('liftingbody.png'))

In [ ]:
iterations=20

## Оценка криволинейного оператора смаза, представленного кривой Безье

In [ ]:
sz = 20
deg = 5
points_real = [(deg-1)*sz//deg-1, sz//deg, sz-1, sz-1]
print(points_real)
#points_real = [sz, 0, sz, sz]
psf_bezier = bezier_psf2(points_real)
plt.imshow(psf_bezier, 'gray')
plt.show()

In [ ]:
lifting_bezier_blurred = convolve2d(liftingbody, psf_bezier, 'same')

lifting_restored = restoration.richardson_lucy(lifting_bezier_blurred, psf_bezier, iterations=iterations)
psnr_py = compare_psnr_crop(liftingbody, lifting_restored)

deconv_bezier = richardson_lucy_matlab(lifting_bezier_blurred, psf_bezier, iterations=iterations, clip=True, useFFT=True, dampar=0.004)
psnr_my = compare_psnr_crop(liftingbody, deconv_bezier['image'])

In [ ]:
show_results(lifting_bezier_blurred, lifting_restored, deconv_bezier['image'],
            titles=['Blurred image', 'Restored with python LR\nPSNR={0}'.format(psnr_py), 'Restored with my LR\nPSNR={0}'.format(psnr_my)])

## Кепстр

In [ ]:
s_n, S_find = find_noise(lifting_bezier_blurred)

In [ ]:
b_clip = 3
N,M = lifting_bezier_blurred.shape
# ( ifft2 (100* log (1+ abs ( fft2 ( I ) ) ) ) ) ;
K = np.fft.ifftn(100*np.log(1+np.abs(np.fft.fftn(lifting_bezier_blurred))))
K_shift = np.fft.fftshift(K)

In [ ]:
c, r = np.unravel_index(np.argmin(K_shift, axis=None), K_shift.shape)
n, m, = K_shift.shape
x2 = np.array([r - n//2, c - m//2])
x1 = x2/2 + np.array([2,-2])
print(x2)

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(np.clip(np.real(K_shift),-1,1))#, vmin=noisy.min(), vmax=noisy.max())
plt.plot(n//2 + x2[0], m//2 + x2[1], 'ro')
plt.title('Кепстр изображения с выделенным минмиумом')
#plt.savefig('pic/{0}-kepstr.png'.format(prefix))
plt.show()

In [ ]:
x0 = np.concatenate((x1, x2))
print(x0)

## Минимизация

In [ ]:
x1, points_real

In [ ]:
res_cg = minimize(partial(funcToMinimizeCurved2, xy2=x2, I_blurred=lifting_bezier_blurred, iterations=iterations, eps=1e-5, dampar=s_n, useFFT=True),
                   options={'disp':True}, x0=x1, method='CG')
print(res_cg)

In [ ]:
deconv_orig = richardson_lucy_matlab(lifting_bezier_blurred, psf_bezier, iterations=iterations, dampar=s_n, useFFT=False)
psnr_orig = compare_psnr(liftingbody, deconv_orig['image'])

psf_init = bezier_psf2(x0)
deconv_init = richardson_lucy_matlab(lifting_bezier_blurred, psf_init, iterations=iterations, dampar=s_n, useFFT=False)
psnr_init = compare_psnr(liftingbody, deconv_init['image'])

points_found = np.concatenate((res_cg['x'], x2))
psf_found = bezier_psf2(points_found)
deconv_found = richardson_lucy_matlab(lifting_bezier_blurred, psf_found, iterations=iterations, dampar=s_n, useFFT=False)
psnr_found = compare_psnr(liftingbody, deconv_found['image'])

show_results(deconv_orig['image'], deconv_init['image'], deconv_found['image'],
             titles=['Restored with true psf\nPSNR={0}\ncoord={1}'.format(psnr_orig, points_real), 
                     'With initial approxiamtion\nPSNR={0}\ncoord={1}'.format(psnr_init, x0),
                     'Minimized error\nPSNR={0}\ncoord={1}'.format(psnr_found, points_found)])

In [ ]:
res_nelder = minimize(partial(funcToMinimizeCurved2, I_blurred=lifting_bezier_blurred, xy2=x2, iterations=iterations, eps=1e-5, dampar=s_n, useFFT=True),
                      x0=x1, method='Nelder-Mead', options={'xatol': 1e-3, 'disp': True})
print(res_nelder)

In [ ]:
# deconv_orig = richardson_lucy_matlab(lifting_bezier_blurred, psf_bezier, iterations=iterations, dampar=s_n, useFFT=False)
# psnr_orig = compare_psnr_crop(liftingbody, deconv_orig['image'])

psf_init = bezier_psf2(x0)
deconv_init = richardson_lucy_matlab(lifting_bezier_blurred, psf_init, iterations=iterations, dampar=s_n, useFFT=False)
psnr_init = compare_psnr_crop(liftingbody, deconv_init['image'])

points_found = np.concatenate((res_nelder['x'], x2))
psf_found = bezier_psf2(points_found)
deconv_found = richardson_lucy_matlab(lifting_bezier_blurred, psf_found, iterations=iterations, dampar=s_n, useFFT=False)
psnr_found = compare_psnr_crop(liftingbody, deconv_found['image'])

show_results(deconv_orig['image'], deconv_init['image'], deconv_found['image'],
             titles=['Restored with true psf\nPSNR={0}\ncoord={1}'.format(psnr_orig.round(2), points_real), 
                     'With initial approxiamtion\nPSNR={0}\ncoord={1}'.format(psnr_init.round(2), x0),
                     'Minimized error\nPSNR={0}\ncoord={1}'.format(psnr_found.round(2), res_nelder['x'])])